In [92]:
import requests
import json
import facebook
import pandas as pd
from datetime import datetime
from datetime import date

class FeedCrawler():
    def __init__(self, token, page_id):
        self.token = token
        self.page_id = page_id
        self.graph = facebook.GraphAPI(access_token = self.token, version="2.12") #套件會用你的token連接到facebook
    def get_result(self):
        result = requests.get('https://graph.facebook.com/v2.9/{}/posts?&access_token={}'.format(self.page_id,self.token))
        return json.dumps(result.json(), indent=2)
    def get_posts_info(self,year, month, date, connection_name = 'posts'):

        time_range = datetime(year, month, date)
        posts = self.graph.get_all_connections(id=self.page_id,
                                       connection_name=connection_name,                                    
                                       since=time_range)
        post_ids =[]
        for ind, post in enumerate(posts):
#             print(ind, post['id'])
            post_ids.append(post['id'])
            
        df =  pd.DataFrame(columns = ['id','created_time','message','post_impressions','post_engaged_users','reply','reactions','interaction','link'])

        for i in post_ids:
            post = self.graph.get_object(id=i, fields='permalink_url,\
                                         comments.limit(0).summary(total_count),\
                                         message,reactions.limit(0).summary(total_count),\
                                         created_time,\
                                         insights.metric(post_impressions,post_engaged_users)')

            if 'message' in post:
                post_info = post['message']
            else:
                post_info = None
            time_stamp = post['created_time']
            post_id = post['id']
            post_impressions = post['insights']['data'][0]['values'][0]['value']
            post_engaged_users = post['insights']['data'][1]['values'][0]['value']
            reply = post['comments']['summary']['total_count']
            reaction = post['reactions']['summary']['total_count']
            interaction = reply + reaction
            link = post['permalink_url']

            test_df = pd.DataFrame(data = {
                'id':post_id,
                'created_time':time_stamp,
                'post_impressions':post_impressions,
                'post_engaged_users':post_engaged_users,
                'message':post_info,
                'reply':reply,
                'reactions':reaction,
                'interaction':interaction,
                'link':link},index=[0])
            df = df.append(test_df, ignore_index=True)
        return df
    def get_comments_info(self,year, month, date, connection_name = 'posts'):
        time_range = datetime(year, month, date)
        posts = self.graph.get_all_connections(id=self.page_id,
                                       connection_name=connection_name,                                    
                                       since=time_range)
        post_ids =[]
        for ind, post in enumerate(posts):
#             print(ind, post['id'])
            post_ids.append(post['id'])
    
    
        df =  pd.DataFrame(columns = ['topics','created_time','message', 'id'])

        for i in post_ids:
            post = self.graph.get_object(id=i, fields='message')
            first_index = post['message'].find('【')
            final_index = post['message'].find('】')
            topics = post['message'][first_index:final_index+1] 

            comments = self.graph.get_connections(id=i, connection_name='comments',limit=1000)
            test_df = pd.DataFrame(comments['data'])
            test_df['topics'] = topics
            df = df.append(test_df, ignore_index=True)
            
        return df
def find_word(df, string):
    s_reply = 0
    s_reaction = 0
    count = 0
    for i in range(len(df)):
        if string in df['message'][i]:
            s_reply = s_reply + df['reply'][i]
            s_reaction = s_reaction + df['reactions'][i]
            count = count + 1
    print('{} count is : {}\n{} reply is : {}\n{} reaction is {}'.format(string,count,string, s_reply,string, s_reaction))

In [95]:
find_word(df, '雷諾')

雷諾 count is : 7
雷諾 reply is : 481
雷諾 reaction is 2852


In [72]:
token = 'EAAMJOXaorZAABAKTcFO7WUTDeXuR8OOafX7pg2GMgKVdpLoUWwuojQ6Q66M60ZBAuIxbzXXRyGTO1Xoq3ZBOpyYZCIasYeQeBjCAR1P2azOWRrIMZB4NAilPuRu7QSZASMAwQ4a0uUaZBszYWZAxDVoCRctuWcVVbEkD2BCzeJMUorVMdnCtqQvOrFoJyBbM1DIAGEPraalvNwZDZD' #貼上前面拿到的Access token
grop_page_id = "1885601821548849"
fanpage_id = '272151260115679'
test = FeedCrawler(token, fanpage_id)

In [73]:
df = test.get_posts_info(2021,1,6)

In [74]:
df.head(50)

,id,created_time,message,post_impressions,post_engaged_users,reply,reactions,interaction,link
0,272151260115679_739391983391602,2021-01-15T16:15:02+0000,⚠【兩周年改版 異常問題修復進度公告】\nHi 我是小橘子~ 我們持續修復問題中😢\n\n稍...,10009,1096,84,186,270,https://www.facebook.com/272151260115679/posts...
1,272151260115679_739240170073450,2021-01-15T10:07:30+0000,🎂【兩周年慶倒數 2 天 五噴 A 車免費送】\nHi 我是小橘子~ 後天就是我們的兩歲生日...,20965,2663,60,390,450,https://www.facebook.com/272151260115679/posts...
2,272151260115679_739095950087872,2021-01-15T04:30:42+0000,🏎【賽道直直撞 S-擎天雷諾傳奇登場】\nHi 我是小橘子~ 傳說中的那台車已經在工坊中登...,16838,1825,50,308,358,https://www.facebook.com/272151260115679/posts...
3,272151260115679_738726340124833,2021-01-14T13:30:01+0000,🎂【兩周年慶倒數 3 天 全 A 車限免即將登場】\nHi 我是小橘子~ 改版後的問題我們陸...,28777,3365,85,513,598,https://www.facebook.com/272151260115679/posts...
4,272151260115679_738748706789263,2021-01-14T12:39:19+0000,⚠【兩周年改版 異常問題修復進度公告】\nHi 我是小橘子~ 抱歉讓大家久等了😢\n目前修正...,16487,1890,118,331,449,https://www.facebook.com/272151260115679/posts...
5,272151260115679_738667240130743,2021-01-14T09:00:41+0000,🏎【歡慶極速兩週年 賽道霸王明天登場】\nHi 我是小橘子~ S 擎天雷諾明天就要登場囉🍊\...,48532,3041,81,438,519,https://www.facebook.com/272151260115679/posts...
6,272151260115679_738558910141576,2021-01-14T07:09:38+0000,🎂【兩周年改版上線 四大車神齊聚歡慶】\nHi 我是小橘子~ 兩周年改版已經正式完成開機囉🍊...,21252,1830,111,330,441,https://www.facebook.com/272151260115679/posts...
7,272151260115679_738561840141283,2021-01-14T04:20:53+0000,⚠【兩周年改版 伺服器維護延遲公告】\nHi 我是小橘子~ 抱歉讓大家久等了😢\n因為這次改...,22127,3014,634,523,1157,https://www.facebook.com/272151260115679/posts...
8,272151260115679_738193743511426,2021-01-13T13:00:00+0000,🎂【兩周年慶倒數 4 天 停機維護迎接新版本】\nHi 我是小橘子~ 明天就是兩周年改版了😍...,21555,2181,91,412,503,https://www.facebook.com/272151260115679/posts...
9,272151260115679_738160390181428,2021-01-13T09:56:42+0000,🏎【歡慶極速兩週年 擎天雷諾電腦版桌布換起來】\nHi 我是小橘子~ S 擎天雷諾後天就要登...,27626,2946,33,601,634,https://www.facebook.com/272151260115679/posts...


In [ ]:
df.sort_values(by = 'created_time', ascending = False)

In [ ]:
df = df.astype({'created_time': 'datetime64[ns]','reply':'int32','reactions':'int32'})

In [ ]:
df[df['created_time'] >= '2020-12-29'].to_excel(r'C:\Users\mgopt4\Desktop\KOL\FB\FB_group_data_{}.xlsx'.format(date.today()), encoding = 'utf-8')

In [ ]:
df_test = df[df['created_time'] >= '2020-12-29'].dropna()

In [ ]:
df_test = df_test.reset_index(drop = True)

In [84]:
s_reply = 0
s_reaction = 0
count = 0
for i in range(len(df)):
    if "小橘子" in df['message'][i]:
        s_reply = s_reply + df['reply'][i]
        s_reaction = s_reaction + df['reactions'][i]
        count = count + 1
print(s_reply,s_reaction,count)

2165 11405 28


In [30]:
token = 'EAAMJOXaorZAABAKTcFO7WUTDeXuR8OOafX7pg2GMgKVdpLoUWwuojQ6Q66M60ZBAuIxbzXXRyGTO1Xoq3ZBOpyYZCIasYeQeBjCAR1P2azOWRrIMZB4NAilPuRu7QSZASMAwQ4a0uUaZBszYWZAxDVoCRctuWcVVbEkD2BCzeJMUorVMdnCtqQvOrFoJyBbM1DIAGEPraalvNwZDZD' #貼上前面拿到的Access token
graph = facebook.GraphAPI(access_token = token, version="2.12") #套件會用你的token連接到facebook


In [34]:
posts = graph.get_all_connections(id='272151260115679',
                                       connection_name='posts',  
                                       fields = 'insights.metric(post_engaged_users)',
                                       since=(2021,1,6))

In [54]:
for i in posts:
    print(i['insights']['data'][0]['values'][0]['value'])
    break

797


In [62]:
for i in posts:
    print(json.dumps(i,indent=4))
    break

{
    "insights": {
        "data": [
            {
                "name": "post_engaged_users",
                "period": "lifetime",
                "values": [
                    {
                        "value": 4039
                    }
                ],
                "title": "Lifetime Engaged Users",
                "description": "Lifetime: The number of unique people who engaged in certain ways with your Page post, for example by commenting on, liking, sharing, or clicking upon particular elements of the post. (Unique Users)",
                "id": "272151260115679_734272213903579/insights/post_engaged_users/lifetime"
            }
        ],
        "paging": {
            "previous": "https://graph.facebook.com/v8.0/272151260115679_734272213903579/insights?access_token=EAAMJOXaorZAABAKTcFO7WUTDeXuR8OOafX7pg2GMgKVdpLoUWwuojQ6Q66M60ZBAuIxbzXXRyGTO1Xoq3ZBOpyYZCIasYeQeBjCAR1P2azOWRrIMZB4NAilPuRu7QSZASMAwQ4a0uUaZBszYWZAxDVoCRctuWcVVbEkD2BCzeJMUorVMdnCtqQvOrFoJyBbM1DIAGEPra